In [2]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Iteration_4').getOrCreate()

Exception: Unable to find py4j in /home/ubuntu/spark-3.2.1-bin-hadoop2.7/python, your SPARK_HOME may not be configured correctly